In [ ]:
import configparser
import ast
import glob
import os
import sys
import subprocess
from subprocess import Popen, PIPE, CalledProcessError

In [ ]:
def execute(cmd):
    popen = subprocess.Popen(cmd, stdout=subprocess.PIPE, universal_newlines=True)
    for stdout_line in iter(popen.stdout.readline, ""):
        yield stdout_line 
    popen.stdout.close()
    return_code = popen.wait()
    if return_code:
        raise subprocess.CalledProcessError(return_code, cmd)

In [ ]:
config = configparser.RawConfigParser(interpolation=configparser.ExtendedInterpolation())
config.read('cytonet.cfg')

In [ ]:
patch_size = 64
load_level = 2
color_channels = 3
experiment_folder = "/root/workspace/data/trained/true_unet2/"

config.set('general', 'patch_size', str(patch_size))
config.set('general', 'load_level', str(load_level))
config.set('general', 'color_channels', str(color_channels))

In [ ]:
filenames = '/root/workspace/data/SVS_train/*.svs'
folders_prefix = 'abc_'
extract_classes=[{'neg': (0,None,2), 'pos' : (1,0,1000), 'back' : (2,100)}]
mask_pattern = "*.png"
extract_folders = []
for file in glob.glob(filenames):
    rm_list = glob.glob(filenames)
    rm_list.remove(file)
    config.set('extraction', 'filenames', str(rm_list))
    i=1
    for extract_class in extract_classes:
        foldername = os.path.join('/root/workspace/data/extraction/', folders_prefix +str(i) + "_" + os.path.splitext(os.path.basename(file))[0])
        extract_folders.append(foldername)
        config.set('extraction', 'output_folder', str(foldername))
        config.set('extraction', 'classes', str(extract_class))
        with open('cytonet.cfg', 'w') as configfile:
            config.write(configfile)
        for path in execute(['python3','ExtractPatches.py']):
            print(path, end="")
        w_file = open(os.path.join(foldername,"testfiles.txt"),"w")
        file_list=[]
        file_list.append(file)
        w_file.write(str(file_list))
        w_file.close()
        i+=1

In [ ]:
files_pattern = "*[!k].png"
classes = {'pos': 0, 'neg' : 0}
config.set('augmentation', 'files_pattern', files_pattern)
config.set('augmentation', 'classes', str(classes))
for folder in extract_folders:
    config.set('augmentation', 'input_folder', folder)
    with open('cytonet.cfg', 'w') as configfile:
            config.write(configfile)
    for path in execute(['python3','Data_Augmentation.py']):
            print(path, end="")

In [ ]:
matrice_files=[]
classes = {'pos' : 1, 'neg': 0}
files_pattern = "*[!mask].png"
config.set('saving', 'files_pattern', files_pattern)
config.set('saving', 'classes', str(classes))
for folder in extract_folders:
    config.set('saving', 'input_folder', folder)
    matrice = os.path.join(folder,"matrice_train.h5")
    config.set('saving', 'output_file', matrice)
    matrice_files.append(matrice)
    with open('cytonet.cfg', 'w') as configfile:
            config.write(configfile)
    for path in execute(['python3','SaveToData.py']):
            print(path, end="")

In [ ]:
classes = {'pos' : 1, 'neg': 0}
batch_size="128"
valid_portion="0.2"
config.set('training', 'batch_size', batch_size)
config.set('training', 'valid_portion', valid_portion)
config.set('training', 'classes', str(classes))
for matrice_file in matrice_files:
    config.set('training', 'matrice_file', matrice_file)
    experiment_folder="/root/workspace/data/trained/" + os.path.basename(os.path.dirname(matrice_file)) +"/" #TODO look / in training
    config.set('training', 'experiment_folder', experiment_folder)
    with open('cytonet.cfg', 'w') as configfile:
            config.write(configfile)
    for path in execute(['python3','Training.py']):
            print(path, end="")

In [ ]:
stride="16"
mask_pattern = "_mask.png"
color_channels = "3"
threshold = "0.5"
output_pattern = "l${general:load_level}_p${general:patch_size}_s${segmentation:stride}_ts${segmentation:threshold}.png"
config.set('segmentation', 'stride', stride)
config.set('segmentation', 'mask_pattern', mask_pattern)
config.set('segmentation', 'color_channels', color_channels)
config.set('segmentation', 'threshold', threshold)
config.set('segmentation', 'output_pattern', output_pattern)
for extract_folder in extract_folders:
    w_file = open(os.path.join(extract_folder,"testfiles.txt"),"r")
    filenames = w_file.readline()
    w_file.close()
    config.set('segmentation', 'filenames', filenames)
    experiment_folder= extract_folder.replace("extraction", "trained")
    config.set('segmentation', 'experiment_folder', experiment_folder)
    with open('cytonet.cfg', 'w') as configfile:
            config.write(configfile)
    for path in execute(['python3','Segmentation_full.py']):
            print(path, end="")

In [ ]:
images = "['/root/workspace/data/SVS_train/*.svs']"
transparency = "110"
display = "False"
config.set('evaluation', 'transparency', transparency)
config.set('evaluation', 'display', display)
config.set('evaluation', 'images', images)
for extract_folder in extract_folders:
    experiment_folder= extract_folder.replace("extraction", "trained")
    config.set('evaluation', 'experiment_folder', experiment_folder)
    with open('cytonet.cfg', 'w') as configfile:
            config.write(configfile)
    for path in execute(['python3','Evaluation.py']):
            print(path, end="")